This notebook holds information gathered from Aladdin Persson's yt tutorial

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
def intersection_over_union(bb_pred, bb_labels,box_format = "midpoint"):
    """ Calculates intersection over Union
    Parameters: bb_pred (tensor) : Prediction of Bounding boxes (BATCH_SIZE,4)
    bb_labels (tensor): Correct labels of BB (BATCH_SIZE, 4)
    box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)
    
    
    Results: 
    tensor : Intersection over union for all examples"""
    #bb_pred shape = (N,4) where N is the bounding boxes
    #bb_labels = (N,4)
    if (box_format == "midpoint"):
        box1_x1 = bb_pred[...,0:1] -  bb_pred[...,2:3]/2
        box1_y1 = bb_pred[...,1:2] - bb_pred[...,3:4]/2
        box1_x2 = bb_pred[...,0:1] + bb_pred[...,2:3]/2
        box1_y2 = bb_pred[...,1:2] + bb_pred[...,3:4]/2
        box2_x1 = bb_labels[...,0:1] -  bb_labels[...,2:3]/2
        box2_y1 = bb_labels[...,1:2] - bb_labels[...,3:4]/2
        box2_x2 = bb_labels[...,0:1] + bb_labels[...,2:3]/2
        box2_y2 = bb_labels[...,1:2] + bb_labels[...,3:4]/2
    elif (box_format == "corners"):
        box1_x1 = bb_pred[...,0:1]
        box1_y1 = bb_pred[...,1:2]
        box1_x2 = bb_pred[...,2:3]
        box1_y2 = bb_pred[...,3:4]
        box2_x1 = bb_labels[...,0:1]
        box2_y1 = bb_labels[...,1:2]
        box2_x2 = bb_labels[...,2:2]
        box2_y2 = bb_labels[...,3:4]
    
    x1 = torch.max(box1_x1,box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box2_x2, box2_x2)
    y2 = torch.min(box2_y2, box2_y2)
    
    
    #clamp[0] is for the case when they do not intersect
    intersection = (x2 - x1).clamp[0] * (y2 - y1).clamp[0]
    
    box1_area = abs((box1_x2 - box1_x1) * (box1_y1 - box1_y2))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y1 - box2_y2))
    
    return intersection/ (box1_area + box2_area - intersection + 1e-6)
    